In [1]:
import pandas as pd

def average_selling_price(prices: pd.DataFrame, units_sold: pd.DataFrame) -> pd.DataFrame:
    # Convert date columns to datetime
    prices['start_date'] = pd.to_datetime(prices['start_date'])
    prices['end_date'] = pd.to_datetime(prices['end_date'])
    units_sold['purchase_date'] = pd.to_datetime(units_sold['purchase_date'])

    # Merge the Prices and UnitsSold DataFrames based on product_id and date range
    merged_df = pd.merge_asof(units_sold.sort_values('purchase_date'), 
                              prices.sort_values('start_date'), 
                              left_on='purchase_date', 
                              right_on='start_date', 
                              by='product_id', 
                              direction='backward')

    # Filter records where purchase_date is within start_date and end_date
    filtered_df = merged_df[(merged_df['purchase_date'] >= merged_df['start_date']) &
                            (merged_df['purchase_date'] <= merged_df['end_date'])]

    # Calculate the total revenue and total units sold for each product_id
    filtered_df['total_price'] = filtered_df['units'] * filtered_df['price']
    total_revenue_df = filtered_df.groupby('product_id')['total_price'].sum().reset_index()
    total_units_sold_df = filtered_df.groupby('product_id')['units'].sum().reset_index()

    # Merge the total revenue and total units sold DataFrames
    result_df = pd.merge(total_revenue_df, total_units_sold_df, on='product_id')

    # Calculate the average selling price
    result_df['average_price'] = result_df['total_price'] / result_df['units']

    # Round the average selling price to 2 decimal places
    result_df['average_price'] = result_df['average_price'].round(2)

    # Handle cases where a product has no sold units
    all_product_ids = pd.DataFrame(prices['product_id'].unique(), columns=['product_id'])
    result_df = pd.merge(all_product_ids, result_df, on='product_id', how='left').fillna(0)
    result_df = result_df[['product_id', 'average_price']]

    return result_df

# Sample data for testing
prices_data = {
    'product_id': [1, 1, 2, 2],
    'start_date': ['2019-02-17', '2019-03-01', '2019-02-01', '2019-02-21'],
    'end_date': ['2019-02-28', '2019-03-22', '2019-02-20', '2019-03-31'],
    'price': [5, 20, 15, 30]
}

units_sold_data = {
    'product_id': [1, 1, 2, 2],
    'purchase_date': ['2019-02-25', '2019-03-01', '2019-02-10', '2019-03-22'],
    'units': [100, 15, 200, 30]
}

# Create DataFrames
prices_df = pd.DataFrame(prices_data)
units_sold_df = pd.DataFrame(units_sold_data)

# Apply the function
result = average_selling_price(prices_df, units_sold_df)

# Display the result
print(result)


   product_id  average_price
0           1           6.96
1           2          16.96
